In [1]:
import importlib
from lib import kpi
from lib import kpi_wh

In [26]:
importlib.reload(kpi)
importlib.reload(kpi_wh)

<module 'lib.kpi_wh' from 'c:\\workspace\\GitHub\\arcsaef\\lib\\kpi_wh.py'>

In [ ]:
data           = kpi.load_data()
rpt_config     = kpi.get_rpt_args()
saef_library   = kpi.get_saef_library()
responses_json = kpi.split_response(data[0])
ppl_collection = kpi.person_construct(responses_json, rpt_config[0])
proj_saef      = kpi.project_construct(responses_json, ppl_collection[0])
# proj_saef      = kpi_wh.projects_construct(data[3])
# ppl_collection = kpi_wh.people_construct(data[3], rpt_config[0])
buckets        = kpi.matched_library(saef_library, ppl_collection[1])
templates      = kpi.load_templates()
meta_bucket    = buckets[0]
bucket         = buckets[1]
ppl_saef       = dict(sorted(ppl_collection[0].items(), key = lambda x: x[1].get('LastName')))
ppl_hash       = ppl_collection[1]
bulk_responses = data[0]
biblio         = data[1]
scopus         = data[2]
rpt_year       = rpt_config[0]
organisations  = rpt_config[1]
org_shortnames = list(organisations.keys())
proj_saef_nohold = proj_saef[proj_saef.Status != 'On hold']

In [39]:
ppl_collection.get('FBC44D1A-F9B8-D744-9B50-F4CD71E237D1')

AttributeError: 'tuple' object has no attribute 'get'

In [6]:
# 1.remove empty rows
bucket = bucket[bucket['id_person'].isna() == False]
# 2. Make parsing more straight forward
for prsn in ppl_saef:
    if ppl_saef[prsn]['Gender'] == 'Non-binary/Gender diverse' or  ppl_saef[prsn]['Gender']  == 'Prefer not to say':
        ppl_saef[prsn]['Gender'] = 'Other'
# 3. Exclude: Nicole Webster, 027E2DEA-DB06-3946-B8C8-E053EF8E09F0
del ppl_saef['027E2DEA-DB06-3946-B8C8-E053EF8E09F0']; del ppl_hash['NicoleWebster']
# 4. Remove leading/trialing whitespace from  blibliographic entry.
biblio['Biblio'] = biblio['Biblio'].str.strip()
# 5. Jump into your pedantry
for prsn in ppl_saef:
    if ppl_saef[prsn]['Position'] != 'PhD Student' and  ppl_saef[prsn]['Position'] != 'Masters Student' and \
       ppl_saef[prsn]['Position'] != 'Honours Student':
        ppl_saef[prsn]['StudentProjectTitle'] = 'Not applicable'

 ### A brief what's where and why
 - Reporting data is from 3 sources;
   - Filemaker: api output (via Postman) is saved as text
     - data/kpi_reporting_yyyymmdd.txt
   - Zotero: api output for reporting year (via Postman) is saved as json
     - data/saef_library_postman_yyyymmdd.json
   - Bibiliography: Zotero bibligraphy output & manual update is saved a xlsx
     - data/all/zot_biblio.xlsx
 - Updating config/reporting.yaml is critical
 - Individual and Organisation cells are run for mid-year and annual reporting
 - Organisation Excel is uused for mid-year

In [27]:
# Individual (annual & mid-year)
for org in org_shortnames[7]:
    for prsn in ppl_saef:
        p = kpi.get_context_idv(org, ppl_saef, prsn, bucket, biblio, rpt_year)
        if len(p) > 0:
            templates['doc_idv'].render(kpi.get_context_idv(org, ppl_saef, prsn, bucket, biblio, rpt_year))
            templates['doc_idv'].save(f"output/{rpt_year}/annual/idv/{org}_{ppl_saef.get(prsn)['LastName']}{ppl_saef.get(prsn)['FirstName']}.docx") 

In [28]:

kpi.get_context_idv('JCU', ppl_saef, 'E77129A9-D1E7-7B4A-AF77-8031D27DF1AE', bucket, biblio, rpt_year)

TypeError: 'NoneType' object is not iterable

In [ ]:
# Organisation (annual & mid-year)
for org in org_shortnames:
    templates['doc_org'].render(kpi.get_context_org(org, organisations, ppl_saef, bucket, biblio, rpt_year, proj_saef_nohold)[0])
    templates['doc_org'].save(f"output/{rpt_year}/midyear/org/{org}.docx")

Report output type: expected string or bytes-like object, got 'NoneType'
Report output type: expected string or bytes-like object, got 'NoneType'


In [11]:
# Organisation Excel (mid-year)
for org in org_shortnames:
    kpi.write_context_org_excel(kpi.get_context_org(org, organisations, ppl_saef, bucket, biblio, rpt_year, proj_saef_nohold), proj_saef, organisations, org)


Report output type: expected string or bytes-like object, got 'NoneType'
Report output type: expected string or bytes-like object, got 'NoneType'


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be remov

In [36]:
print(ppl_saef.get('81E17FF3-F45C-074F-9A24-83BC9189D10E')['Projects'])

None


In [ ]:
print(ppl_saef.get())

None
